# Geração de instâncias

In [57]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from udgp import M1, M2, M4, M5, Instance

In [58]:
env = gp.Env()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-07-25


In [65]:
with open("3nir.pdb") as ifile:
    system = "".join([x for x in ifile])

In [66]:
import py3Dmol

In [70]:
view = py3Dmol.view(width=800, height=800)
view.addModelsAsFrames(system)
view.setBackgroundColor("#FAFAFA", 1.0)
view.setStyle({"cartoon": {'color': 'spectrum'}})
# view.setStyle(
#         {
#             "stick": {"radius": 0.1, "color": "#cbd5e1"},
#             "sphere": {"scale": 0.2, "color": "#60a5fa"},
#         }
#     )
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [64]:
# seed: 1234567
# instance = Instance.artificial_molecule(50, freq=True, seed=1234567)
instance = Instance.lj_cluster(50, freq=True)
# instance = Instance.c60(freq=True)
instance.view_input(bg_color="#FAFAFA", alpha=1, width=800, height=800)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Testes

In [8]:
instance.reset()
m = M4(
        instance, 
        # nx=4,
        env=env,
        # relaxed=True,
    )
m.optimize(log=True)
print(instance.is_solved())
print(m.Runtime)
instance.view()

Set parameter LogToConsole to value 1
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 302 rows, 195 columns and 780 nonzeros
Model fingerprint: 0x3503897d
Model has 15 quadratic constraints
Variable types: 135 continuous, 30 integer (30 binary)
Semi-Variable types: 30 continuous, 0 integer
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e-03, 2e+00]
  RHS range        [5e-01, 1e+01]
Presolve removed 91 rows and 45 columns
Presolve time: 0.00s
Presolved: 466 rows, 240 columns, 1080 nonzeros
Presolved model has 60 bilinear constraint(s)
Variable types: 195 continuous, 45 integer (45 binary)

Root relaxation: objective 1.000000e+00, 167 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [4]:
t = []

# for i in range(5):
for seed in [1234567, 123456, 12345]:
    print(seed)
    instance = Instance.artificial_molecule(6, freq=False, seed=seed)
    m = M1(
        instance, 
        # nx=4, 
        env=env,
        # relaxed=True,
    )
    m.optimize(log=False)
    print(instance.is_solved())
    t.append(m.Runtime)

np.mean(t)


1234567

Interrupt request received
distância 1.580078125 não está na lista: [1.5  1.5  1.5  1.5  1.5  2.45 2.45 2.45 2.45 2.45 2.87 2.87 2.87 2.87
 3.46]
frequências: [0 0 0 0 1 1 1 1 1 1 1 1 1 1 1]
erro mínimo: 1.5 -> 0.05078125
False
123456


In [6]:
instance.is_solved()

False

In [14]:
print(instance.dists)
print(instance.freqs)

[1.12 1.15 1.82]
[11  0  0]


# Heurística

In [11]:
from IPython.display import clear_output

instance.reset()

previous_a = []

time = 0

i = 1
while not instance.is_solved():
# while instance.fixed_n < instance.n:
    clear_output(wait=True)
    print("=================")
    print(f"TENTATIVA {i}")
    print("=================")
    i += 1

    instance.reset()
    m_core = M4(instance, nx=4, env=env)
    for a_indices in previous_a:
        m_core.addConstr(
            gp.quicksum(m_core.a[i, j, k] for i, j, k in a_indices) 
            <= len(a_indices) - 1
        )
    m_core.optimize()

    if m.Status == GRB.INFEASIBLE:
        previous_a = []

    time += m_core.Runtime

    previous_a.append(instance.a_indices.tolist())
    
    # instance.reset_with_core("mock")
    
    print("CORE")

    broken = False
    while instance.fixed_n < instance.n and not broken:
        print(f"  {instance.fixed_n} átomos")

        solved = False
        tries = 0

        while not solved:
            tries += 1
    
            m = M4(instance, nx=1, env=env)

            solved = m.optimize()
        
            time += m.Runtime
    
            if m.Status == GRB.INFEASIBLE or tries > 20:
                broken = True
                # previous_a.append([(i, j, k) for i, j, k in instance.a_indices])
                break

print(time)

instance.view()


TENTATIVA 24
Modelo inviável.
CORE
  1 átomos
  2 átomos
  3 átomos
  4 átomos


  5 átomos


In [9]:
import numpy as np
xxx = np.array([0.72, 3.99, 9.39, 60.52, 114.57])
xxx = xxx*60
xxx = xxx*27.35/(0.72*60)
xxx = xxx + np.random.normal(100,100,5)
xxx

array([ 245.64799542,  261.11746201,  613.61174006, 2421.56062166,
       4539.25102258])

In [55]:
rng = np.random.default_rng()
rng.random()  

0.013400657263876847